# DATAFRAMES

In [139]:
from pyspark.sql import SparkSession

In [140]:
spark = SparkSession.builder.appName("Dtaframes").getOrCreate()

In [141]:
df = spark.read.json("companies.json")

In [142]:
df.show()

+--------+-----+
|    name|sales|
+--------+-----+
|   apple| null|
|  google| 2000|
|facebook|  150|
+--------+-----+



In [143]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- sales: long (nullable = true)



In [144]:
df.columns

['name', 'sales']

In [145]:
df.describe().show()

+-------+------+-----------------+
|summary|  name|            sales|
+-------+------+-----------------+
|  count|     3|                2|
|   mean|  null|           1075.0|
| stddev|  null|1308.147545195113|
|    min| apple|              150|
|    max|google|             2000|
+-------+------+-----------------+



In [146]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType

In [148]:
schema = [StructField("sales", IntegerType(), True), StructField("name", StringType(), True)]

In [149]:
end_schema = StructType(fields = schema)

In [150]:
df = spark.read.json("companies.json", schema = end_schema )

In [151]:
df.printSchema()

root
 |-- sales: integer (nullable = true)
 |-- name: string (nullable = true)



In [152]:
df.show()

+-----+--------+
|sales|    name|
+-----+--------+
| null|   apple|
| 2000|  google|
|  150|facebook|
+-----+--------+



In [155]:
type(df['sales'])

pyspark.sql.column.Column

In [156]:
df.select("sales").show()

+-----+
|sales|
+-----+
| null|
| 2000|
|  150|
+-----+



In [161]:
df.head(2)

[Row(sales=None, name='apple'), Row(sales=2000, name='google')]

In [162]:
df.select(["sales","name"]).show()

+-----+--------+
|sales|    name|
+-----+--------+
| null|   apple|
| 2000|  google|
|  150|facebook|
+-----+--------+



In [164]:
df.withColumn("new_column", df['sales']*2).show()

+-----+--------+----------+
|sales|    name|new_column|
+-----+--------+----------+
| null|   apple|      null|
| 2000|  google|      4000|
|  150|facebook|       300|
+-----+--------+----------+



In [165]:
df.withColumnRenamed("sales", "sales_year").show()

+----------+--------+
|sales_year|    name|
+----------+--------+
|      null|   apple|
|      2000|  google|
|       150|facebook|
+----------+--------+



In [167]:
df.createOrReplaceTempView("companies")

In [168]:
query = spark.sql("select * from companies where sales > 180")

In [169]:
query.show()

+-----+------+
|sales|  name|
+-----+------+
| 2000|google|
+-----+------+



In [171]:
df = spark.read.csv("apl.csv", inferSchema= True, header = True)

In [172]:
df.show()

+-------------------+---------+---------+---------+---------+---------+---------+
|               Date|     Open|     High|      Low|    Close|Adj Close|   Volume|
+-------------------+---------+---------+---------+---------+---------+---------+
|2010-01-04 00:00:00|    30.49|30.642857|    30.34|30.572857|26.466835|123432400|
|2010-01-05 00:00:00|30.657143|30.798571|30.464285|30.625713|26.512596|150476200|
|2010-01-06 00:00:00|30.625713|30.747143|30.107143|30.138571|26.090879|138040000|
|2010-01-07 00:00:00|    30.25|30.285715|29.864286|30.082857|26.042646|119282800|
|2010-01-08 00:00:00|30.042856|30.285715|29.865715|30.282858|26.215786|111902700|
|2010-01-11 00:00:00|     30.4|30.428572|29.778572|30.015715|25.984528|115557400|
|2010-01-12 00:00:00|29.884285|29.967142|29.488571|29.674286|25.688946|148614900|
|2010-01-13 00:00:00|29.695715|30.132856|29.157143|30.092857|26.051304|151473000|
|2010-01-14 00:00:00|30.015715|30.065714|29.860001|29.918571|25.900436|108223500|
|2010-01-15 00:0

In [173]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)



In [174]:
df.head(2)

[Row(Date=datetime.datetime(2010, 1, 4, 0, 0), Open=30.49, High=30.642857, Low=30.34, Close=30.572857, Adj Close=26.466835, Volume=123432400),
 Row(Date=datetime.datetime(2010, 1, 5, 0, 0), Open=30.657143, High=30.798571, Low=30.464285, Close=30.625713, Adj Close=26.512596, Volume=150476200)]

In [177]:
df.filter("Close < 28").select(["Open","Close"]).show()

+---------+---------+
|     Open|    Close|
+---------+---------+
|28.725714|27.437143|
|27.481428|27.818571|
|27.987143|    27.98|
|28.104286|27.435715|
|27.518572|27.922857|
|27.955715|27.731428|
|27.984285|27.874287|
+---------+---------+



In [178]:
df.where((df["Close"] < 30) | (df["Open"] < 30)).select(["Open","Close"]).show()

+---------+---------+
|     Open|    Close|
+---------+---------+
|29.884285|29.674286|
|29.695715|30.092857|
|30.015715|29.918571|
|30.132856|29.418571|
|29.761429|30.719999|
|30.297142|29.724285|
|29.540001|    28.25|
|    28.93|    29.01|
|29.421429|    29.42|
|29.549999|29.697144|
|29.275715|28.469999|
|28.725714|27.437143|
|27.481428|27.818571|
|27.987143|    27.98|
|27.881428|28.461428|
|28.104286|27.435715|
|27.518572|27.922857|
|27.955715|27.731428|
|28.059999|28.027143|
|27.984285|27.874287|
+---------+---------+
only showing top 20 rows



In [179]:
df.where((df["Close"] < 30) & ~(df["Open"] < 30)).select(["Open","Close"]).show()

+---------+---------+
|     Open|    Close|
+---------+---------+
|30.015715|29.918571|
|30.132856|29.418571|
|30.297142|29.724285|
+---------+---------+



In [180]:
result = df.where(df["Low"] == 30.34).collect()

In [181]:
result

[Row(Date=datetime.datetime(2010, 1, 4, 0, 0), Open=30.49, High=30.642857, Low=30.34, Close=30.572857, Adj Close=26.466835, Volume=123432400)]

In [182]:
result[0]

Row(Date=datetime.datetime(2010, 1, 4, 0, 0), Open=30.49, High=30.642857, Low=30.34, Close=30.572857, Adj Close=26.466835, Volume=123432400)

In [184]:
result[0].asDict()['Close']

30.572857

In [185]:
df = spark.read.csv("sales.csv", inferSchema = True, header=True)

In [186]:
df.show()

+-------+---------+-----+
|company|empleoyee|sales|
+-------+---------+-----+
|   GOOG|     John|  200|
|   GOOG|    Maria|  120|
|   GOOG|     Josh|  340|
|   MSFT|  Michael|  600|
|   MSFT|   Andrew|  124|
|   MSFT|    Peter|  243|
|     FB|     Sara|  870|
|     FB|    Dhrew|  350|
|   APPL| Victoria|  250|
|   APPL|    Oscar|  130|
|   APPL|   George|  750|
|   APPL|     Ivan|  350|
+-------+---------+-----+



In [187]:
df.printSchema()

root
 |-- company: string (nullable = true)
 |-- empleoyee: string (nullable = true)
 |-- sales: integer (nullable = true)



In [188]:
df.groupBy("company").sum().show()

+-------+----------+
|company|sum(sales)|
+-------+----------+
|   APPL|      1480|
|   GOOG|       660|
|     FB|      1220|
|   MSFT|       967|
+-------+----------+



In [189]:
df.groupBy("company").mean().show()

+-------+-----------------+
|company|       avg(sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [190]:
df.groupBy("company").max().show()

+-------+----------+
|company|max(sales)|
+-------+----------+
|   APPL|       750|
|   GOOG|       340|
|     FB|       870|
|   MSFT|       600|
+-------+----------+



In [191]:
df.groupBy("company").count().show()

+-------+-----+
|company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [192]:
df.agg({"sales":"sum"}).show()

+----------+
|sum(sales)|
+----------+
|      4327|
+----------+



In [193]:
group = df.groupBy("company")

In [195]:
group.agg({"sales":"sum"}).show()

+-------+----------+
|company|sum(sales)|
+-------+----------+
|   APPL|      1480|
|   GOOG|       660|
|     FB|      1220|
|   MSFT|       967|
+-------+----------+



In [196]:
from pyspark.sql.functions import countDistinct, avg, stddev, format_number

In [197]:
df.select(countDistinct("sales")).show()

+---------------------+
|count(DISTINCT sales)|
+---------------------+
|                   11|
+---------------------+



In [198]:
df.select(countDistinct("sales").alias("distinct values")).show()

+---------------+
|distinct values|
+---------------+
|             11|
+---------------+



In [199]:
df.select(stddev("sales").alias("stddev sales")).show()

+------------------+
|      stddev sales|
+------------------+
|250.08742410799007|
+------------------+



In [201]:
sales_stddev = df.select(stddev("sales").alias("stddev sales"))

In [202]:
sales_stddev.select(format_number('stddev sales', 2).alias("stddev sales formatted")).show() 

+----------------------+
|stddev sales formatted|
+----------------------+
|                250.09|
+----------------------+



In [203]:
df.orderBy("sales").show()

+-------+---------+-----+
|company|empleoyee|sales|
+-------+---------+-----+
|   GOOG|    Maria|  120|
|   MSFT|   Andrew|  124|
|   APPL|    Oscar|  130|
|   GOOG|     John|  200|
|   MSFT|    Peter|  243|
|   APPL| Victoria|  250|
|   GOOG|     Josh|  340|
|     FB|    Dhrew|  350|
|   APPL|     Ivan|  350|
|   MSFT|  Michael|  600|
|   APPL|   George|  750|
|     FB|     Sara|  870|
+-------+---------+-----+



In [204]:
df.orderBy(df["sales"].desc()).show()

+-------+---------+-----+
|company|empleoyee|sales|
+-------+---------+-----+
|     FB|     Sara|  870|
|   APPL|   George|  750|
|   MSFT|  Michael|  600|
|     FB|    Dhrew|  350|
|   APPL|     Ivan|  350|
|   GOOG|     Josh|  340|
|   APPL| Victoria|  250|
|   MSFT|    Peter|  243|
|   GOOG|     John|  200|
|   APPL|    Oscar|  130|
|   MSFT|   Andrew|  124|
|   GOOG|    Maria|  120|
+-------+---------+-----+



In [206]:
df = spark.read.csv("nullvalues.csv", inferSchema = True, header=True) 

In [207]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- sales: integer (nullable = true)
 |-- customer: integer (nullable = true)



In [208]:
df.show()

+----+-----+-----+--------+
|  Id| name|sales|customer|
+----+-----+-----+--------+
|emp1| John| null|       3|
|emp2| null| null|    null|
|emp3| null|  345|    null|
|emp4|Cindy|  456|       4|
+----+-----+-----+--------+



In [209]:
df.na.drop().show()

+----+-----+-----+--------+
|  Id| name|sales|customer|
+----+-----+-----+--------+
|emp4|Cindy|  456|       4|
+----+-----+-----+--------+



In [216]:
df.na.drop(thresh=1).show()

+----+-----+-----+--------+
|  Id| name|sales|customer|
+----+-----+-----+--------+
|emp1| John| null|       3|
|emp2| null| null|    null|
|emp3| null|  345|    null|
|emp4|Cindy|  456|       4|
+----+-----+-----+--------+



In [217]:
df.na.drop(how = 'any').show()

+----+-----+-----+--------+
|  Id| name|sales|customer|
+----+-----+-----+--------+
|emp4|Cindy|  456|       4|
+----+-----+-----+--------+



In [218]:
df.na.drop(how = 'all').show()

+----+-----+-----+--------+
|  Id| name|sales|customer|
+----+-----+-----+--------+
|emp1| John| null|       3|
|emp2| null| null|    null|
|emp3| null|  345|    null|
|emp4|Cindy|  456|       4|
+----+-----+-----+--------+



In [219]:
df.na.drop(subset = 'sales').show()

+----+-----+-----+--------+
|  Id| name|sales|customer|
+----+-----+-----+--------+
|emp3| null|  345|    null|
|emp4|Cindy|  456|       4|
+----+-----+-----+--------+



In [220]:
df.na.drop(subset = 'customer').show()

+----+-----+-----+--------+
|  Id| name|sales|customer|
+----+-----+-----+--------+
|emp1| John| null|       3|
|emp4|Cindy|  456|       4|
+----+-----+-----+--------+



In [221]:
df.na.fill(0).show()

+----+-----+-----+--------+
|  Id| name|sales|customer|
+----+-----+-----+--------+
|emp1| John|    0|       3|
|emp2| null|    0|       0|
|emp3| null|  345|       0|
|emp4|Cindy|  456|       4|
+----+-----+-----+--------+



In [222]:
df.na.fill(0, subset=["sales"]).show()

+----+-----+-----+--------+
|  Id| name|sales|customer|
+----+-----+-----+--------+
|emp1| John|    0|       3|
|emp2| null|    0|    null|
|emp3| null|  345|    null|
|emp4|Cindy|  456|       4|
+----+-----+-----+--------+



In [ ]:
df.na.fill(0, subset=["sales"]).show()

In [223]:
from pyspark.sql.functions import mean

In [224]:
media = df.select(mean(df["sales"])).collect()

In [227]:
media[0][0]

400.5

In [228]:
df.na.fill(media[0][0], ["sales"]).show()

+----+-----+-----+--------+
|  Id| name|sales|customer|
+----+-----+-----+--------+
|emp1| John|  400|       3|
|emp2| null|  400|    null|
|emp3| null|  345|    null|
|emp4|Cindy|  456|       4|
+----+-----+-----+--------+



In [262]:
from pyspark.sql.functions import dayofmonth, dayofyear, month, year, weekofyear, format_number, date_format, concat_ws

In [231]:
df = spark.read.csv("apl.csv", inferSchema= True, header = True)

In [232]:
df.show()

+-------------------+---------+---------+---------+---------+---------+---------+
|               Date|     Open|     High|      Low|    Close|Adj Close|   Volume|
+-------------------+---------+---------+---------+---------+---------+---------+
|2010-01-04 00:00:00|    30.49|30.642857|    30.34|30.572857|26.466835|123432400|
|2010-01-05 00:00:00|30.657143|30.798571|30.464285|30.625713|26.512596|150476200|
|2010-01-06 00:00:00|30.625713|30.747143|30.107143|30.138571|26.090879|138040000|
|2010-01-07 00:00:00|    30.25|30.285715|29.864286|30.082857|26.042646|119282800|
|2010-01-08 00:00:00|30.042856|30.285715|29.865715|30.282858|26.215786|111902700|
|2010-01-11 00:00:00|     30.4|30.428572|29.778572|30.015715|25.984528|115557400|
|2010-01-12 00:00:00|29.884285|29.967142|29.488571|29.674286|25.688946|148614900|
|2010-01-13 00:00:00|29.695715|30.132856|29.157143|30.092857|26.051304|151473000|
|2010-01-14 00:00:00|30.015715|30.065714|29.860001|29.918571|25.900436|108223500|
|2010-01-15 00:0

In [234]:
df.select(dayofmonth(df["Date"])).show()

+----------------+
|dayofmonth(Date)|
+----------------+
|               4|
|               5|
|               6|
|               7|
|               8|
|              11|
|              12|
|              13|
|              14|
|              15|
|              19|
|              20|
|              21|
|              22|
|              25|
|              26|
|              27|
|              28|
|              29|
|               1|
+----------------+
only showing top 20 rows



In [235]:
df.select(dayofyear(df["Date"])).show()

+---------------+
|dayofyear(Date)|
+---------------+
|              4|
|              5|
|              6|
|              7|
|              8|
|             11|
|             12|
|             13|
|             14|
|             15|
|             19|
|             20|
|             21|
|             22|
|             25|
|             26|
|             27|
|             28|
|             29|
|             32|
+---------------+
only showing top 20 rows



In [236]:
new_df = df.withColumn("Year", (year(df["Date"])))

In [237]:
new_df.show()

+-------------------+---------+---------+---------+---------+---------+---------+----+
|               Date|     Open|     High|      Low|    Close|Adj Close|   Volume|Year|
+-------------------+---------+---------+---------+---------+---------+---------+----+
|2010-01-04 00:00:00|    30.49|30.642857|    30.34|30.572857|26.466835|123432400|2010|
|2010-01-05 00:00:00|30.657143|30.798571|30.464285|30.625713|26.512596|150476200|2010|
|2010-01-06 00:00:00|30.625713|30.747143|30.107143|30.138571|26.090879|138040000|2010|
|2010-01-07 00:00:00|    30.25|30.285715|29.864286|30.082857|26.042646|119282800|2010|
|2010-01-08 00:00:00|30.042856|30.285715|29.865715|30.282858|26.215786|111902700|2010|
|2010-01-11 00:00:00|     30.4|30.428572|29.778572|30.015715|25.984528|115557400|2010|
|2010-01-12 00:00:00|29.884285|29.967142|29.488571|29.674286|25.688946|148614900|2010|
|2010-01-13 00:00:00|29.695715|30.132856|29.157143|30.092857|26.051304|151473000|2010|
|2010-01-14 00:00:00|30.015715|30.065714|29

In [238]:
result = new_df.groupBy("Year").mean()

In [239]:
result.show()

+----+------------------+------------------+------------------+------------------+------------------+--------------------+---------+
|Year|         avg(Open)|         avg(High)|          avg(Low)|        avg(Close)|    avg(Adj Close)|         avg(Volume)|avg(Year)|
+----+------------------+------------------+------------------+------------------+------------------+--------------------+---------+
|2018|189.11143456573691|190.99410358167344|187.18350615936254| 189.0534267649403|184.07218519123506| 3.402006454183267E7|   2018.0|
|2015|120.17575393253965|121.24452385714291| 118.8630954325397|120.03999980555547|110.69666290873012|5.1849404365079366E7|   2015.0|
|2013| 67.58973367460315| 68.23413245238099| 66.89244326190476| 67.51926860317461| 59.77199219047618|          1.016087E8|   2013.0|
|2014| 92.21983011507933| 93.01266438888888| 91.47553844047621| 92.26465432539683| 83.65277066269844| 6.315273055555555E7|   2014.0|
|2019|207.86908675793657|  209.831706626984|206.27162703174605| 208.2

In [244]:
result = new_df.groupBy("Year").mean().select(["Year", "avg(Open)", "avg(Close)"])

In [245]:
result.show()

+----+------------------+------------------+
|Year|         avg(Open)|        avg(Close)|
+----+------------------+------------------+
|2018|189.11143456573691| 189.0534267649403|
|2015|120.17575393253965|120.03999980555547|
|2013| 67.58973367460315| 67.51926860317461|
|2014| 92.21983011507933| 92.26465432539683|
|2019|207.86908675793657| 208.2559521944445|
|2020|300.64314603225813| 301.6202412096774|
|2012| 82.37896010000001| 82.29281706400006|
|2016|104.50777772619044|104.60400786904763|
|2010| 37.13680271825399|37.120351420634925|
|2011|52.008775376984104|52.000617904761896|
|2017|150.44490045816727|150.55107548605574|
+----+------------------+------------------+



In [246]:
result.select('Year', format_number("avg(Open)", 4).alias("Avg Open"), format_number("avg(Close)", 4).alias("Avg Close")).orderBy(result["Year"]).show()

+----+--------+---------+
|Year|Avg Open|Avg Close|
+----+--------+---------+
|2010| 37.1368|  37.1204|
|2011| 52.0088|  52.0006|
|2012| 82.3790|  82.2928|
|2013| 67.5897|  67.5193|
|2014| 92.2198|  92.2647|
|2015|120.1758| 120.0400|
|2016|104.5078| 104.6040|
|2017|150.4449| 150.5511|
|2018|189.1114| 189.0534|
|2019|207.8691| 208.2560|
|2020|300.6431| 301.6202|
+----+--------+---------+



In [254]:
columns = df.columns
prefix = "new_"

In [256]:
[prefix + c for c in columns]

['new_Date',
 'new_Open',
 'new_High',
 'new_Low',
 'new_Close',
 'new_Adj Close',
 'new_Volume']

In [264]:
df.select( [ concat_ws("", c).alias(prefix + c) for c in columns]  ).show()

+-------------------+---------+---------+---------+---------+-------------+----------+
|           new_Date| new_Open| new_High|  new_Low|new_Close|new_Adj Close|new_Volume|
+-------------------+---------+---------+---------+---------+-------------+----------+
|2010-01-04 00:00:00|    30.49|30.642857|    30.34|30.572857|    26.466835| 123432400|
|2010-01-05 00:00:00|30.657143|30.798571|30.464285|30.625713|    26.512596| 150476200|
|2010-01-06 00:00:00|30.625713|30.747143|30.107143|30.138571|    26.090879| 138040000|
|2010-01-07 00:00:00|    30.25|30.285715|29.864286|30.082857|    26.042646| 119282800|
|2010-01-08 00:00:00|30.042856|30.285715|29.865715|30.282858|    26.215786| 111902700|
|2010-01-11 00:00:00|     30.4|30.428572|29.778572|30.015715|    25.984528| 115557400|
|2010-01-12 00:00:00|29.884285|29.967142|29.488571|29.674286|    25.688946| 148614900|
|2010-01-13 00:00:00|29.695715|30.132856|29.157143|30.092857|    26.051304| 151473000|
|2010-01-14 00:00:00|30.015715|30.065714|29